In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14,8
RANDOM_SEED = 42

In [2]:
data = pd.read_csv('churn_data.csv')

In [3]:
data.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,1,1,3,29.85,29.85,0
1,5575-GNVDE,34,1,2,0,4,56.95,1889.50,0
2,3668-QPYBK,2,1,1,1,4,53.85,108.15,1
3,7795-CFOCW,45,0,2,0,1,42.30,1840.75,0
4,9237-HQITU,2,1,1,1,3,70.70,151.65,1


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['customerID'] = label_encoder.fit_transform(data['customerID'])

In [5]:
data.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,1,0,1,1,3,29.85,29.85,0
1,3962,34,1,2,0,4,56.95,1889.50,0
2,2564,2,1,1,1,4,53.85,108.15,1
3,5535,45,0,2,0,1,42.30,1840.75,0
4,6511,2,1,1,1,3,70.70,151.65,1


In [6]:
#Create X and y features
columns = data.columns.tolist()
columns = [c for c in columns if c not in ["Churn"]]
target = "Churn"
state = np.random.RandomState(42)
X = data[columns]
y = data[target]
print(X.shape)
print(y.shape)

(7043, 8)
(7043,)


## Isolation Forest

In [7]:
from sklearn.ensemble import IsolationForest

In [37]:
model = IsolationForest(contamination=0.1, random_state=42)
# fit
model.fit(X_train, y_train)

# predict
y_pred = model.predict(X_test)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [38]:
res = model.fit(X)
y_train_pred = res.predict(X_train)
y_train_pred[:10]

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


array([ 1,  1,  1,  1,  1,  1,  1,  1, -1,  1])

In [40]:
y_train_pred_final = pd.DataFrame({'Churn':y_train.values, 'Churn_Prob':y_train_pred})
y_train_pred_final['CustID'] = y_train.index
y_train_pred_final.head()

,Churn,Churn_Prob,CustID
0,0,1,1695
1,0,1,1095
2,0,1,3889
3,0,1,3667
4,0,1,2902


In [41]:
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

,Churn,Churn_Prob,CustID,predicted
0,0,1,1695,1
1,0,1,1095,1
2,0,1,3889,1
3,0,1,3667,1
4,0,1,2902,1


In [42]:
from sklearn import metrics

In [43]:
# Confusion matrix 
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
print(confusion)

[[ 404 3231]
 [  83 1212]]


In [44]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

0.327789046653144
